In [1]:
import pandas as pd
import torch
import pandas as pd
import os
import librosa
import contextlib
from mutagen.mp3 import MP3
from datasets import load_dataset, Value, Features, Audio, Dataset, concatenate_datasets, DatasetDict
from sklearn.preprocessing import minmax_scale
import numpy as np
import evaluate
from transformers import set_seed
torch.cuda.empty_cache()
set_seed(42)
import re
# Details: https://huggingface.co/docs/diffusers/optimization/fp16#enable-cudnn-autotuner
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True
from transformers import pipeline
os.cpu_count()

In [6]:
from datasets import load_dataset
from transformers import WhisperForConditionalGeneration, WhisperProcessor
import torch
from evaluate import load

In [2]:
from transformers import WhisperForConditionalGeneration, WhisperProcessor
from transformers import pipeline
from transformers.utils import is_flash_attn_2_available

processor = WhisperProcessor.from_pretrained("openai/whisper-large-v2")
model = WhisperForConditionalGeneration.from_pretrained("RafatK/Swahili-Whisper-Largev2-Decodis_FT", torch_dtype=torch.float16).to("cuda")
model.generation_config.input_ids = model.generation_config.forced_decoder_ids
model.generation_config.forced_decoder_ids = None
forced_decoder_ids = processor.get_decoder_prompt_ids(language="swahili", task="transcribe")

pipe = pipeline(
  "automatic-speech-recognition",
  model=model,
    processor = "openai/whisper-large-v2",
    tokenizer = "openai/whisper-large-v2",
    feature_extractor = "openai/whisper-large-v2",
  chunk_length_s=15,
  device=device,
  model_kwargs={"attn_implementation": "flash_attention_2"} if is_flash_attn_2_available() else {"attn_implementation": "sdpa"},
  generate_kwargs = {
                       'num_beams':5,
                       'max_new_tokens':440,
                       'early_stopping':True,
                       }
)
text_output = pipe("audio.wav")['text']


In [3]:
import datasets

features = Features(
    {   # "file_name": Value('string'),
        "path": Audio(sampling_rate=16000),
        "transcription": Value("string"),
       
    }
)
in_house = datasets.load_dataset("csv", data_files={"test": "openslr/pi_test.csv"}, delimiter="\t", features = features)["test"]

In [4]:
# from datasets import load_dataset
# fleurs = load_dataset("google/fleurs", "sw_ke", split="test")

In [5]:
processor = WhisperProcessor.from_pretrained("openai/whisper-large-v2")
model = WhisperForConditionalGeneration.from_pretrained("whisper_swhaili", torch_dtype=torch.float16).to("cuda")
forced_decoder_ids = processor.get_decoder_prompt_ids(language="swahili", task="transcribe")

In [6]:
import torch
from transformers.utils import is_flash_attn_2_available
from transformers import pipeline
from datasets import load_dataset

device = "cuda:0" if torch.cuda.is_available() else "cpu"

pipe = pipeline(
  "automatic-speech-recognition",
  model=model,
    processor = "openai/whisper-large-v2",
    tokenizer = "openai/whisper-large-v2",
    feature_extractor = "openai/whisper-large-v2",
  chunk_length_s=15,
  device=device,
    model_kwargs={"attn_implementation": "flash_attention_2"} if is_flash_attn_2_available() else {"attn_implementation": "sdpa"},
    generate_kwargs = {

                       'repetition_penalty':1.8,
                       'num_beams':5,
                       'max_new_tokens':444,
                       'early_stopping':True,

                       }
)

In [8]:
fleur_path_test = []
fleur_trans_test = []
for i in range(len(fleurs)):
    s = fleurs[i]['path']
    t = fleurs[i]['audio']['path']
    path  = "\\".join(s.split("\\")[:-1])+"\\"+t
    fleur_path_test.append(path)
    fleur_trans_test.append(fleurs[i]["raw_transcription"])

In [7]:
pred = []
for i in range(len(fleur_path_test)):
    print(i)
    prediction = pipe(fleur_path_test[i]['path']['path'])['text']
    pred.append(prediction)

In [23]:
labels = fleur_trans_test

In [26]:
from whisper_normalizer.basic import BasicTextNormalizer

normalizer = BasicTextNormalizer()

for i in range(len(pred)):
    pred[i] = normalizer(pred[i])
    labels[i] = normalizer(labels[i])

In [8]:
bleu = evaluate.load("wer")
results = bleu.compute(predictions=pred, references=labels)
print(results)